In [1]:
import threading
import time
import random

BUFFER_SIZE = 5
buffer = []

lock = threading.Lock()
not_full = threading.Condition(lock)
not_empty = threading.Condition(lock)

In [2]:
def camera():
    for i in range(10):
        time.sleep(random.uniform(0.4, 1.0))

        with not_full:
            while len(buffer) >= BUFFER_SIZE:
                not_full.wait()
            buffer.append(f"Frame-{i}")
            print(f"[Camera] Produced Frame-{i} | Buffer size: {len(buffer)}")
            not_empty.notify()

In [3]:
def satellite():
    for _ in range(10):
        with not_empty:
            while len(buffer) == 0:
                not_empty.wait()
            frame = buffer.pop(0)
            print(f"    [Satellite] Consumed {frame} | Buffer size: {len(buffer)}")
            not_full.notify()
        time.sleep(random.uniform(0.6, 1.2))

In [4]:
producer = threading.Thread(target=camera)
consumer = threading.Thread(target=satellite)

producer.start()
consumer.start()

producer.join()
consumer.join()

print("\nSimulation finished.")


[Camera] Produced Frame-0 | Buffer size: 1
    [Satellite] Consumed Frame-0 | Buffer size: 0
[Camera] Produced Frame-1 | Buffer size: 1
    [Satellite] Consumed Frame-1 | Buffer size: 0
[Camera] Produced Frame-2 | Buffer size: 1
    [Satellite] Consumed Frame-2 | Buffer size: 0
[Camera] Produced Frame-3 | Buffer size: 1
[Camera] Produced Frame-4 | Buffer size: 2
    [Satellite] Consumed Frame-3 | Buffer size: 1
[Camera] Produced Frame-5 | Buffer size: 2
    [Satellite] Consumed Frame-4 | Buffer size: 1
[Camera] Produced Frame-6 | Buffer size: 2
[Camera] Produced Frame-7 | Buffer size: 3
    [Satellite] Consumed Frame-5 | Buffer size: 2
[Camera] Produced Frame-8 | Buffer size: 3
    [Satellite] Consumed Frame-6 | Buffer size: 2
[Camera] Produced Frame-9 | Buffer size: 3
    [Satellite] Consumed Frame-7 | Buffer size: 2
    [Satellite] Consumed Frame-8 | Buffer size: 1
    [Satellite] Consumed Frame-9 | Buffer size: 0

Simulation finished.
